## **Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install numpy==1.23.5 librosa==0.10.0 jams --quiet

In [2]:
import os
import numpy as np
import librosa
import jams
from tqdm import tqdm

In [12]:
BASE_PATH = "/content/drive/MyDrive/Automatic Guitar Transcription/Data Set/GuitarSet"
AUDIO_DIR = os.path.join(BASE_PATH, "audio", "mono_mic")
ANNOTATION_DIR = os.path.join(BASE_PATH, "annotation")
OUTPUT_DIR = "/content/drive/MyDrive/Automatic Guitar Transcription/processed_data/cnn_ctc"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## **Mel Spectrogram**

In [13]:
def extract_melspec(audio_path, sr=16000, n_mels=128, hop_length=512):
    y, _ = librosa.load(audio_path, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    return mel_db.T  # [T x F]

## **Note Events**

In [14]:
def extract_note_events(jams_path):
    jam = jams.load(jams_path)
    note_events = []
    for ann in jam.search(namespace='note_midi'):
        for note in ann:
            if note.duration > 0:  # sıfır süreli notaları alma
                note_events.append((note.time, note.value))
    return sorted(note_events)

In [15]:
PITCH_MIN = 40
PITCH_MAX = 88
pitch_to_class = {p: i+1 for i, p in enumerate(range(PITCH_MIN, PITCH_MAX+1))}  # 0 = blank (CTC için)

In [16]:
def create_ctc_target(note_events):
    seq = []
    prev = -1
    for onset, pitch in note_events:
        pitch = int(round(pitch))  # 🔧 YUVARLA: 51.03 → 51
        if pitch in pitch_to_class:
            cls = pitch_to_class[pitch]
            if cls != prev:
                seq.append(cls)
                prev = cls
    return seq

## **Process & Save**

In [17]:
def get_all_ids(annotation_dir):
    return sorted([f.replace(".jams", "") for f in os.listdir(annotation_dir) if f.endswith(".jams")])

In [18]:
for file_id in tqdm(get_all_ids(ANNOTATION_DIR)):
    try:
        audio_path = os.path.join(AUDIO_DIR, file_id + "_mic.wav")
        jams_path = os.path.join(ANNOTATION_DIR, file_id + ".jams")
        if not os.path.exists(audio_path):
            continue

        mel = extract_melspec(audio_path)
        note_events = extract_note_events(jams_path)
        label_seq = create_ctc_target(note_events)

        if len(label_seq) < 2:
            print(f"⚠️ {file_id} label sequence too short, skipping.")
            continue

        np.savez_compressed(
            os.path.join(OUTPUT_DIR, f"{file_id}.npz"),
            mel=mel.astype(np.float32),
            label=np.array(label_seq, dtype=np.int32),
            input_length=np.array([mel.shape[0]], dtype=np.int32),
            target_length=np.array([len(label_seq)], dtype=np.int32)
        )
    except Exception as e:
        print(f"❌ {file_id} hatası: {e}")

100%|██████████| 360/360 [02:48<00:00,  2.14it/s]


In [10]:
for file_id in get_all_ids(ANNOTATION_DIR)[:5]:
    jams_path = os.path.join(ANNOTATION_DIR, file_id + ".jams")
    note_events = extract_note_events(jams_path)
    print(f"🎼 {file_id} içinde {len(note_events)} adet note var.")
    print(note_events[:5])

🎼 00_BN1-129-Eb_comp içinde 133 adet note var.
[(0.04881632653064116, 51.036803851780434), (0.04979138322005383, 65.04964902107794), (0.05271655328806446, 62.091407982224176), (0.5199954648526273, 51.03245930740626), (0.7220362811791574, 58.15072802770881)]
🎼 00_BN1-129-Eb_solo içinde 71 adet note var.
[(0.008544217687131095, 56.17883958529012), (0.009972789115749947, 46.00026427896257), (0.015052154195132061, 51.21675296624269), (0.18695691609980258, 53.18443623507049), (0.18770521541955532, 57.73527096026936)]
🎼 00_BN1-147-Gb_comp içinde 151 adet note var.
[(0.007574603174475669, 53.1619922313278), (0.051452154194976174, 61.073789222207836), (0.054331972788986604, 58.04470383096172), (0.05521632653062625, 65.03915655011332), (0.05562448979594592, 41.96868462567043)]
🎼 00_BN1-147-Gb_solo içinde 89 adet note var.
[(0.008685714285661561, 46.259766221590944), (0.43605532879814746, 49.18344741611403), (0.6376879818594716, 47.207536501006864), (0.8586630385487979, 46.22347977022038), (1.06